In [59]:
from ortools.sat.python.cp_model import CpModel, CpSolver
import numpy as np

sudoku_table = [
    [None, 1, None, 7, None, None, None, None, None],
    [None, None, 2, None, None, None, None, 7, 8],
    [None, 3, 4, None, None, None, None, 6, 1],
    [None, None, None, None, None, 2, 9, None, None],
    [None, None, None, None, 3, 1, 2, 5, None],
    [None, None, None, 5, None, None, 6, None, None],
    [9, None, 6, None, None, 7, None, None, 2],
    [None, 7, None, None, None, None, None, None, None],
    [None, 2, 8, None, 9, None, None, None, None],
]

In [60]:
model = CpModel()
variables = {}

COLUMNS = len(sudoku_table)
ROWS = len(sudoku_table[0])

for row in range(ROWS):
    for col in range(COLUMNS):
        variables[row, col] = model.new_int_var(lb=1, ub=9, name=f"cell_{row}_{col}")

# force initial values
for row in range(ROWS):
    for col in range(COLUMNS):
        cell_value = sudoku_table[row][col]
        if cell_value is not None:
            model.add(variables[row, col] == cell_value)


# column wise different
for col in range(COLUMNS):
    model.add_all_different([variables[row, col] for row in range(ROWS)])

# row wise different
for row in range(ROWS):
    model.add_all_different([variables[row, col] for col in range(COLUMNS)])

# 3x3 all different
for row in range(0, ROWS, 3):
    for col in range(0, COLUMNS, 3):
        three_by_three = list(
            (x, y) for x in range(row, row + 3) for y in range(col, col + 3)
        )
        model.add_all_different([variables[i] for i in three_by_three])

model.minimize(0)

In [61]:
solver = CpSolver()
solver.solve(model)
print(solver.status_name())
solution = np.zeros_like(sudoku_table)
for row in range(ROWS):
    for col in range(COLUMNS):
        solution[row, col] = solver.value(variables[row, col])

solution

OPTIMAL


array([[8, 1, 5, 7, 6, 3, 4, 2, 9],
       [6, 9, 2, 4, 1, 5, 3, 7, 8],
       [7, 3, 4, 9, 2, 8, 5, 6, 1],
       [3, 5, 1, 6, 7, 2, 9, 8, 4],
       [4, 6, 9, 8, 3, 1, 2, 5, 7],
       [2, 8, 7, 5, 4, 9, 6, 1, 3],
       [9, 4, 6, 1, 5, 7, 8, 3, 2],
       [5, 7, 3, 2, 8, 4, 1, 9, 6],
       [1, 2, 8, 3, 9, 6, 7, 4, 5]], dtype=object)